## **s1reader + ISCE3 + dolphin**

In [1]:
import os
import re
import sys
import glob
from pathlib import Path
# Insert local scripts directory to the system path
notebook_dir = Path().resolve()
scripts_dir = str(notebook_dir.parent / 'scripts')
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir)


# Input workspace and meta data location
WORKSPACE = "/mnt/e/InSAR/Stuttgart/ISCE3"          # The workspace prepared for ISCE2 processing
RAW_LOCATION = "/mnt/e/InSAR/Stuttgart/RAW"         # The location of Sentinel-1 TOPS data
LOCAL_DEM_DIR = "/mnt/e/DEM/SRTMGL1/"               # The location which you stored all the local SRTM GL1 DEM tiles

# Then automatically set up the orbit and auxliary calibration files directory
ORBIT_DIR = os.path.join(WORKSPACE, "ORB")
os.makedirs(ORBIT_DIR, exist_ok=True)

The following steps: Download orbit data -> Download and stitch dem -> Generate DEM config -> Run stack processing

In [2]:
# Locate all SAFE files in the directory
safe_files = glob.glob(os.path.join(RAW_LOCATION, 'S1*_IW_SLC*'))
if not safe_files:
    print(f"No SAFE files found in directory: {RAW_LOCATION}")

print(f"Found {len(safe_files)} SAFE files")

Found 15 SAFE files


<b> 1. Download DEM </b>

In [3]:
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

def _get_orbit_list() -> list[str]:
    """
    从 ASF（Alaska Satellite Facility）服务器获取 Sentinel-1 卫星轨道文件列表。
    
    该函数从 ASF 的轨道文件存储库获取所有可用的精密轨道文件（.EOF 格式）列表。
    这些轨道文件包含卫星的精确位置信息，用于 InSAR 数据处理中的几何校正和配准。
    
    Returns:
        list[str]: 轨道文件名列表（.EOF 格式）。如果获取失败，返回空列表。
    """
    url_root = "https://s1qc.asf.alaska.edu/aux_poeorb"

    try:
        session = _create_session()
        response = session.get(url_root, timeout=30)
        response.raise_for_status()

        # print(response.text)
        # Extract all .EOF files from the HTML
        orbit_files = re.findall(r'href="([^"]*\.EOF)"', response.text)
        print(f"Found {len(orbit_files)} orbit files.")
        return orbit_files

    except Exception as e:
        print(f"Failed to get orbit list: {e}")
        return []

def _create_session() -> requests.Session:
    """
    创建一个带有重试策略的 requests 会话。
    
    配置了自动重试机制，用于处理网络请求中的常见错误（如服务器错误、超时等），
    提高下载轨道文件的可靠性。
    
    Returns:
        requests.Session: 配置好重试策略的会话对象。
    """
    session = requests.Session()

    # Setup retry strategy
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
    )

    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    return session

# Download Precise Orbit
from datetime import datetime, timedelta
orbit_list = _get_orbit_list()
if not orbit_list:
    print("Failed to get orbit file list from ASF server")


def _download_orbit_file_asf(orbit_file: str, output_path: os.PathLike) -> bool:
    url_root = "https://s1qc.asf.alaska.edu/aux_poeorb"
    file_url = f"{url_root}/{orbit_file}"

    try:
        session = _create_session()
        response = session.get(file_url, stream=True, timeout=60)
        response.raise_for_status()

        # Get file size
        total_size = int(response.headers.get('content-length', 0))
        downloaded_size = 0

        with open(output_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    downloaded_size += len(chunk)

                    # Show progress
                    if total_size > 0:
                        progress = (downloaded_size / total_size) * 100
                        print(f"\r  Progress: {progress:.1f}%", end='', flush=True)

        print()  # New line after progress
        return True

    except Exception as e:
        print(f"\n  Download failed: {e}")
        # Clean up partially downloaded file
        if output_path.exists():
            output_path.unlink()
        return False

print(f"Start downloading orbit files from ASF...")
# For each SAFE file, download the corresponding orbit file
for safe_file in safe_files:
    print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
    print(f"Processing: {os.path.basename(safe_file)}")
    
    # Extract mission and timestamp from SAFE 
    basename = os.path.basename(safe_file)

    # Remove .SAFE or .zip extension
    if basename.endswith('.SAFE'):
        basename = basename[:-5]
    elif basename.endswith('.zip'):
        basename = basename[:-4]

    # Split by underscores
    parts = basename.split('_')

    if len(parts) < 6:
        print(f"  Invalid filename format: {basename}")
        continue

    # Mission identifier (S1A or S1B)
    mission = parts[0]

    # Timestamps are typically at positions 5 and 6
    # Format: YYYYMMDDTHHMMSS
    start_time_str = parts[5]
    stop_time_str = parts[6]

    # Parse timestamps
    try:
        start_time = datetime.strptime(start_time_str, '%Y%m%dT%H%M%S')
        stop_time = datetime.strptime(stop_time_str, '%Y%m%dT%H%M%S')
    except ValueError:
        print(f"  Could not parse timestamps from: {basename}")
        continue

    if not mission or not start_time:
        print("  Could not parse SAFE filename, skipping")
        continue

    # Calculate date range for orbit search (one day before and after)
    date1 = (start_time - timedelta(days=1)).strftime('%Y%m%d')
    date2 = (start_time + timedelta(days=1)).strftime('%Y%m%d')

    # Find matching orbit file
    mission_short = mission[-1]
    # Search for matching orbit file
    for orbit_file in orbit_list:
    # Check if file matches mission and contains both dates
        if (f"S1{mission_short}" in orbit_file and
            date1 in orbit_file and date2 in orbit_file):
            # Check if orbit file already exists in orbit directory
            orbit_path = os.path.join(ORBIT_DIR, orbit_file)
            if os.path.exists(orbit_path):
                print(f"  Orbit file already exists: {orbit_file}")
                continue
            # Download orbit file
            success = _download_orbit_file_asf(orbit_file, orbit_path)

            if success:
                print(f"  Successfully downloaded: {orbit_file}")
            else:
                print(f"  Failed to download orbit file for: {os.path.basename(safe_file)}")
    
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished downloading orbit files.")

Found 10955 orbit files.
Start downloading orbit files from ASF...
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */

Processing: S1A_IW_SLC__1SDV_20240103T053454_20240103T053522_051938_06467F_A0C6.zip
  Orbit file already exists: S1A_OPER_AUX_POEORB_OPOD_20240123T070858_V20240102T225942_20240104T005942.EOF
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */

Processing: S1A_IW_SLC__1SDV_20240115T053454_20240115T053522_052113_064C75_0023.zip
  Orbit file already exists: S1A_OPER_AUX_POEORB_OPOD_20240204T070755_V20240114T225942_20240116T005942.EOF
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */

Processing: S1A_IW_SLC__1SDV_20240127T053454_20240127T053522_052288_065258_FDD9.zip
  Orbit file already exists: S1A_OPER_AUX_POEORB_OPOD_20240216T070746_V20240126T225942_20240128T005942.EOF
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

<b>2. Download DEM</b>

In [4]:
# Download DEM
import zipfile
# First, customize a function to detect the lat/lon region of the SLCs
lon_list = []
lat_list = []
polygons = []
print(f"Start preparing DEM ...")
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
for safe_file in safe_files:
    if safe_file.endswith(".SAFE"):
        try:
            # For unzipped SAFE files, find the KML file in the preview directory
            kml_file = glob.glob(os.path.join(safe_file, "preview", f"map-overlay.kml"))
            if kml_file:
                with open(kml_file[0], 'r', encoding='utf-8') as kml:  # Fixed variable name and encoding
                    content = kml.read()
        except Exception as e:
            raise IOError(f"Error extracting KML from SAFE file {safe_file}: {e}")
        
    elif safe_file.endswith(".zip"):
        # For zipped SAFE files, directly read the KML file from inside the ZIP
        try:
            with zipfile.ZipFile(safe_file, 'r') as zip_ref:
                # Search for the map-overlay.kml file in the preview directory
                kml_entries = [entry for entry in zip_ref.namelist() if "preview/map-overlay.kml" in entry]
                if kml_entries:
                    # Read the KML content directly from the ZIP file
                    with zip_ref.open(kml_entries[0]) as kml_file:
                        content = kml_file.read().decode('utf-8', errors='ignore')
        except Exception as e:
            raise IOError(f"Error processing ZIP file {safe_file}: {e}")

    coord_pattern = r'<coordinates[^>]*>(.*?)</coordinates>'
    coord_matches = re.findall(coord_pattern, content, re.DOTALL | re.IGNORECASE)

    for coord_block in coord_matches:
        points = []
        coord_text = coord_block.strip()
        coord_text = re.sub(r'\s+', ' ', coord_text)
        coord_triplets = coord_text.split()

        for coord_str in coord_triplets:
            if ',' in coord_str:
                parts = coord_str.split(',')
                if len(parts) >= 2:
                    try:
                        lon = float(parts[0])
                        lat = float(parts[1])
                        if -180 <= lon <= 180 and -90 <= lat <= 90:
                            points.append((lon, lat))
                    except ValueError:
                        pass

    if points:
        # Remove duplicate closing point if exists
        if len(points) > 1 and points[0] == points[-1]:
            points = points[:-1]
        polygons.append(points)
    
if polygons and len(polygons) > 0:
    lons = [pt[0] for pt in polygons[0]]
    lats = [pt[1] for pt in polygons[0]]
    lon_list.extend(lon for lon in lons)
    lat_list.extend(lat for lat in lats)

# Find the common lat/lon region of the SLCs
DEM_REGION = (int(min(lat_list)), int(max(lat_list)) + 1, int(min(lon_list)), int(max(lon_list)) + 1)
print(f"DEM_REGION: {DEM_REGION}")
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")

Start preparing DEM ...
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */

DEM_REGION: (47, 51, 7, 12)
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */



In [ ]:
from mosaic_dem import prepare_dem_core
dem_output_prefix = os.path.join(WORKSPACE, "DEM", f"demLat_N{DEM_REGION[0]:02d}_N{DEM_REGION[1]:02d}_Lon_E{DEM_REGION[2]:03d}_E{DEM_REGION[3]:03d}")
os.makedirs(os.path.dirname(dem_output_prefix), exist_ok=True)
os.chdir(LOCAL_DEM_DIR)
prepare_dem_core(
    bbox=DEM_REGION,
    source="srtm",
    dem_dir=LOCAL_DEM_DIR,
    local_only=True,
    output=dem_output_prefix,
    height="ellipsoidal",
    sample="3601"
)
dem_output = dem_output_prefix + ".dem.wgs84"
print("* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */\n")
print(f"Finished downloading and mosaicing DEM - {dem_output}")

Region: Latitude 51°N–47°N, Longitude 7°E–12°E
DEM source: SRTM
Directory: /mnt/e/DEM/SRTMGL1/ 
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */

Found 20 tiles:
  N50E007.hgt
  N50E008.hgt
  N50E009.hgt
  N50E010.hgt
  N50E011.hgt
  N49E007.hgt
  N49E008.hgt
  N49E009.hgt
  N49E010.hgt
  N49E011.hgt
  N48E007.hgt
  N48E008.hgt
  N48E009.hgt
  N48E010.hgt
  N48E011.hgt
  N47E007.hgt
  N47E008.hgt
  N47E009.hgt
  N47E010.hgt
  N47E011.hgt
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */

Mosaicing 20 tiles into 4x5 grid
Successfully mosaiced DEM with shape (14400, 18000)
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */



/home/wang_xuanxuan/tools/miniforge/envs/opera/lib/python3.12/site-packages/osgeo/gdal.py:330: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Saved GeoTIFF: /mnt/e/InSAR/Stuttgart/ISCE3/DEM/demLat_N47_N51_Lon_E007_E012_orthometric.tif
Generated ISCE2 XML metadata: /mnt/e/InSAR/Stuttgart/ISCE3/DEM/demLat_N47_N51_Lon_E007_E012.xml
* - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - */

Converting heights from EGM96 to WGS84 ellipsoidal heights


In [ ]:
from update_cslc_config import main as update_cslc_config_main
config_file = os.path.join(WORKSPACE, "s1_cslc.yaml")
product_dir = os.path.join(WORKSPACE, "PRODUCT")
scrath_dir = os.path.join(WORKSPACE, "SCRATCH")
sas_output = os.path.join(product_dir, "sas_output.json")
iargs = [config_file,
         "--safe-dir", RAW_LOCATION, 
         "--orbit-dir", ORBIT_DIR, 
         "--dem-file", dem_output, 
         "--product-path", product_dir, 
         "--scratch-path", scrath_dir, 
         "--sas-file", sas_output, 
         '--gpu-enabled']   

update_cslc_config_main(iargs)
